In [1]:
# Check your current folder:
import filepath 
local_folder = filepath.output_folder 
print(local_folder)

D:\Users\lilig\Desktop\SMS-NewData\


In [2]:

import infrastructure 
import pygsheets
import time
import pandas as pd 
import numpy as np 
import datetime as dt 
import send_email 
import os
from os import listdir
from os.path import isfile, join
from datetime import timedelta,datetime 

# import publisher information 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
rxmgref = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
publisher  = rxmgref.worksheet('title','Publisher Configurations').get_as_df()
#arschedule  = rxmgref.worksheet('title','Flow Draft').get_as_df()
#arschedule = arschedule[arschedule['Date Ended']=='']
arschedule  = infrastructure.get_smartsheet('ar_sms')
arschedule = arschedule[arschedule['Day'].notna()]
arschedule = arschedule[arschedule['Date Ended'].isna()]
arschedule= arschedule.drop('Status', axis=1)
publisher_unique = publisher[publisher['PUBID']!=''][['DP.DS or DP.sV','PUBID','DMA']].drop_duplicates()
arschedule = arschedule.merge(publisher_unique, left_on = 'DP.DS', right_on = 'DP.DS or DP.sV', how = 'left') 

# import offer sheet 
offer_sms = infrastructure.get_smartsheet('offers_sms')
offer_sms = offer_sms[offer_sms['Hitpath Offer ID'].isna() == False]
# import offer sheet from email 
offer_email_raw_data=infrastructure.get_smartsheet('offers_email')
offer_email=offer_email_raw_data[['Hitpath Offer ID','Scheduling Name','Operational Status','Payout']]
offer_email_raw_data = offer_email_raw_data.add_suffix(' from email')

# import mamba 
mamba = gc.open_by_url('https://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=534096291') 
new_mamba  = mamba.worksheet('title','New Mamba')
schedule = new_mamba.get_as_df()

# define today 
today = pd.to_datetime(dt.datetime.today())

#get holiday restriction list 
holiday_dict = {}
def holiday_restriction(url): 
    holiday_restriction = gc.open_by_url(url)
    holiday_restriction_sheet  = holiday_restriction.worksheet('title','SMS Drop Restrictions')
    holiday_restriction_df = holiday_restriction_sheet.get_as_df()
    holiday_restriction_df.columns = holiday_restriction_df.iloc[5]
    holiday_restriction_df1= holiday_restriction_df.iloc[7:]
    holiday_restriction_df1['HitPath ID'] = holiday_restriction_df1['HitPath ID'].astype(str).str.split('.',expand = True)[0]
    temp = holiday_restriction_df1.set_index('HitPath ID').iloc[:,5:]
    for i in temp.columns:
        df_for_date = temp[i]
        df_for_date1 = df_for_date[(df_for_date.str.upper()=='NO DROPS') | (df_for_date.str.upper()=='X')]
        i = pd.to_datetime(i)
        holiday_dict[i] = df_for_date1.index.tolist()
    return holiday_dict
holiday_dict = holiday_restriction('https://docs.google.com/spreadsheets/d/127S2S6Krl7KZJzBlxKJg8L2HrZqWAoJ8XAPGEFGhOVc/edit?gid=832173927#gid=832173927')
#holiday_dict = holiday_restriction('https://docs.google.com/spreadsheets/d/1USZQ_CIpfEV-NDH8K9oVy-hy3cTh4uaD0SuksRCtEq8/edit#gid=1068286208')



# La nina 
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1060654066')
lanina_sheet =  lanina.worksheet('title','La Nina (Current)')
lanina_df = lanina_sheet.get_as_df()

# DA payout 
dapayout = gc.open_by_url('https://docs.google.com/spreadsheets/d/1iUxuhBWag0Pamg-GVLpfiJo-ENc2AQzdw3uYhAXVEcQ/edit#gid=1010710644')
offer9287 =  dapayout.worksheet('title','Endurance').get_as_df()
offer9287.columns = offer9287.iloc[0].reset_index()[0].tolist()
offer9287 = offer9287[1:]
offer7878 =  dapayout.worksheet('title','ADT').get_as_df()
offer7878.columns = offer7878.iloc[0].reset_index()[0].tolist()
offer7878 = offer7878[1:]

# offer wall 
offerwall = infrastructure.get_smartsheet("ow_sms")


D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
D:\Users\lilig\A

In [3]:
holiday_dict

{Timestamp('2024-07-02 00:00:00'): ['13294'],
 Timestamp('2024-07-03 00:00:00'): ['13294'],
 Timestamp('2024-07-04 00:00:00'): ['13504', '13559', '13294', '13475'],
 Timestamp('2024-07-05 00:00:00'): ['13504', '13294'],
 Timestamp('2024-07-06 00:00:00'): ['13504', '13294'],
 Timestamp('2024-07-07 00:00:00'): ['13504', '13294']}

In [4]:
# # get holiday restriction list 
# holiday_dict = {}
# def holiday_restriction(url): 
#     holiday_restriction = gc.open_by_url(url)
#     holiday_restriction_sheet  = holiday_restriction.worksheet('title','SMS Drop Restrictions')
#     holiday_restriction_df = holiday_restriction_sheet.get_as_df()
#     holiday_restriction_df.columns = holiday_restriction_df.iloc[3]
#     holiday_restriction_df1= holiday_restriction_df.iloc[4:]
#     holiday_restriction_df1['HitPath ID'] = holiday_restriction_df1['HitPath ID'].astype(str).str.replace('.0','')
#     temp = holiday_restriction_df1.set_index('HitPath ID').iloc[:,4:]
#     for i in temp.columns:
#         df_for_date = temp[i]
#         df_for_date1 = df_for_date[(df_for_date.str.upper()=='NO DROPS') | (df_for_date.str.upper()=='X')]
#         i = pd.to_datetime(i)
#         holiday_dict[i] = df_for_date1.index.tolist()
#     return holiday_dict
# holiday_dict = holiday_restriction('https://docs.google.com/spreadsheets/d/1lbVedR2EMjRnUhBzyyn47vm6GjI9uKpIKF6p72nmsrg/edit#gid=832173927')

In [5]:
# check whether there's any duplicated  CCID in La Nina 
lanina_df[lanina_df['Reporting Content ID'].duplicated()]['Reporting Content ID'].tolist()

['',
 '',
 '',
 '',
 'SC.MBC.454041',
 '13194.TF.N3G.454832',
 '13349.TF.N3G.455021',
 '13349.TF.N3G.455022',
 '13349.TF.MFA.455021',
 '13349.TF.MFA.455022',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '..',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ''

# Update in La Nina

In [6]:
offer_sms_merge = offer_sms[['Hitpath Offer ID','Status','Vertical']].drop_duplicates()
lanina1  = lanina_df.merge(offer_sms_merge[['Hitpath Offer ID','Status','Vertical']], how = 'left',  left_on = 'OfferIDs',right_on ='Hitpath Offer ID',copy = False )
lanina1
lanina1['Status'] = lanina1['Status'].fillna('')
offer_status_list = ['Offer Status'] +  lanina1['Status'].tolist()
lanina_sheet.update_col(21, offer_status_list)
lanina1['Vertical'] = lanina1['Vertical'].fillna('')
offer_vertical_list = ['Offer Vertical'] +  lanina1['Vertical'].tolist()
lanina_sheet.update_col(22, offer_vertical_list)
# Define the condition and new value
condition_column_index = 4 # Index of the column containing the condition
condition_value = 'Approved - Pipeline Testing'  # Condition to match in the column
condition_column_index_2 = 21
condition_value_2 = 'Passed Test - In Production'
target_column_index = 4 # Index of the column to update
new_value = 'Approved - Passed'  # New value to update


# Iterate over dataframe
for index, row in lanina1.iterrows():
    # Check your condition
    if (row['Content Approval Status'].strip() == 'Pending') & ((row['Status'] == condition_value_2) |(row['Status'] == 'Passed Test - Dupe Bucket')  ):
        # access Data using 
        cell = lanina_sheet.cell((index+2, 4))
        cell.value = 'Approved - Passed' 
        cell.update()


In [7]:
rxpcontent_sheet = lanina.worksheet('title','La Nina for RXP ETL')
rxpcontent = lanina_sheet.get_as_df()

#
flc_url = "https://cf.freedomlender.co/"
mbc_url = "https://cf.mybenefitsclub.com/"
dss_url = "https://members.dollar-sensei.com/"
hzb_url = "https://cf.horizonbenefits.org/"
uaa_url = "https://members.unitedamericanalliance.com/"
svt_url = "https://members.solvent.tools/"
prc_url = "https://members.parentsresourceconnection.co/"
rxp_network_offer_id = pd.read_csv(filepath.rxp_network_offer_id)
#rxp_network_offer_id['Hitpath Offer ID'] = rxp_network_offer_id['ain'].str.extract( '(\d+)')
rxp_network_offer_id['Hitpath Offer ID'] = rxp_network_offer_id['meta_data'].str.extract( '(\d+)')
rxp_network_offer_id['network_offer_id'] = rxp_network_offer_id['network_offer_id'].str.extract( '(\d+)')


# change int to str 
rxpcontent['OfferIDs']  = rxpcontent['OfferIDs'].astype(str).str.replace('.0','')
rxp_network_offer_id['network_offer_id']  = rxp_network_offer_id['network_offer_id'].astype(str).str.replace('.0','')
rxp_network_offer_id['Hitpath Offer ID'] = rxp_network_offer_id['Hitpath Offer ID'].astype(str).str.replace('.0','')

# merge table 
rxpcontent = rxpcontent.merge(rxp_network_offer_id[['Hitpath Offer ID','network_offer_id']] , how = 'left', left_on = 'OfferIDs', right_on = 'Hitpath Offer ID',copy = False)
# production offer wasn't shown in the rxp 
#rxpcontent['network_offer_id'] = rxpcontent['network_offer_id'].fillna('')
rxpcontent.loc[rxpcontent['Hitpath Offer ID'].isna() == False, 'network_offer_id' ] = rxpcontent['network_offer_id'].astype(str).str.replace('nan','')
rxpcontent['Hitpath Offer ID'] = rxpcontent['Hitpath Offer ID'].fillna('')
# offer we couldn't find in rxp
off_hit = offer_sms['Hitpath Offer ID'].astype(str).str.split(".",expand=True)[0].tolist()
print(rxpcontent[(rxpcontent['network_offer_id'].isna())&(rxpcontent['Offer Status'].isin(['Passed Test - In Production','Passed Test - Dupe Bucket'])) & (rxpcontent['OfferIDs'].isin(off_hit)) ]['OfferIDs'].unique())
rest_url =  "o/?cfid="+rxpcontent['OfferIDs'] + "&oid="+rxpcontent['network_offer_id'] +"&emailaddress={{emailaddress}}&firstname={{firstname}}&lastname={{lastname}}&phonenumbermobile={{phonenumbermobile}}&cityname={{cityname}}&postalzipcode={{postalzipcode}}"



rxpcontent['Offer Target URL']  = ''
rxpcontent.loc[(rxpcontent['Type'] == 'FLC')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = flc_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'MBC')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = mbc_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'DSS')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = dss_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'HZB')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = hzb_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'UAA')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = uaa_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'SVT')& (rxpcontent['Channel'] == 'SC') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = svt_url + rest_url
rxpcontent.loc[(rxpcontent['Type'] == 'PRC')& (rxpcontent['Channel'] == 'TF') & (rxpcontent['network_offer_id'].isna() == False), 'Offer Target URL'] = prc_url + rest_url




[]


In [8]:
rxp_network_offer_id

,meta_data,network_offer_id,Hitpath Offer ID
0,NaN,nan,nan
1,"{hitpath_ids"": [11714]}""",3090,11714
2,"{hitpath_ids"": [6182]}""",13,6182
3,"{hitpath_ids"": [11612]}""",3090,11612
4,"{hitpath_ids"": [11468]}""",23683,11468
...,...,...,...
92,"{hitpath_ids"": [11646]}""",4447,11646
93,"{hitpath_ids"": [6974]}""",1804,6974
94,"{hitpath_ids"": [12066]}""",nan,12066
95,NaN,4167,nan


In [9]:
#rxpcontent['Offer Target URL'].unique()
pattern = r'({{.*?}})'
rxpcontent['Content_rxp'] = rxpcontent['Content']
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{.*?\|First_name\|.*?}}','{{First_name}}',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{.*?\|first_name\|.*?}}','{{first_name}}',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{.*?\|city\|.*?}}','{{city}}',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{.*?\|region\|.*?}}','{{region}}',regex = True)
#rxpcontent['Content'].str.extractall(pattern)[0].unique()
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{region}}|{{Region}}','%%provincestatename%%',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace(r'{{first_name}}|{{First_name}}','%%first_name%%',regex = True)
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{today_month}}','%%date:F|today%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{today_dd_mm_yy}}','%%date:d-m-y|today%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{city}}','%%cityname%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{today_mm_dd_yy}}','%%date:m-d-y|today%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{tomorrow_month}}','%%date:F|tomorrow%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{tomorrow_mon_dd_yy}}','%%date:M, d Y|tomorrow%%')
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('{{random|Erica|Ally|Tom|Grant|Mark|Julie}}','Tom')

# replace the stop to end with ""
rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].str.replace('STOP to end','')
rxpcontent['Content_rxp'].str.extractall(pattern)[0].unique()

rxpcontent['Content_rxp'] = rxpcontent['Content_rxp'].astype('str')
rxpcontent['Offer Target URL'] = rxpcontent['Offer Target URL'].fillna('')
rxpcontent['Offer Target URL'] = rxpcontent['Offer Target URL'].astype('str')
# Function to replace the text
def replace_target_url(row):
    if (row['Offer Target URL'] != '') & (row['Offer Target URL'] != np.nan):
        return row['Content_rxp'].replace('{{offer_target_url}}', row['Offer Target URL'])
    return ''

    
rxpcontent['Content_rxp_with_url'] = rxpcontent.apply(replace_target_url, axis=1)
rxpcontent = rxpcontent.drop(columns = ['Drop Count','Content_rxp'])
rxpcontent['Link Type'] = 'jump page'
rxpcontent = rxpcontent[['index', 'Reporting Content ID', 'Long Code Content ID',
       'Content Approval Status', 'Content Status in Platform', 'Type (Pitch)',
       'OfferIDs', 'Advertiser', 'OfferAbbrv', 'Channel', 'Platform', 'Type',
       'Allocation Period (Date Added)', 'Piece Number', 'Copywriter',
       'Date Restrictions', 'Character Count', 'Content','Content_rxp_with_url', 'Review by SMS Team',
       'Link & "STOP to end"', 'Offer Status', 'Offer Vertical', 'Hitpath Offer ID',
       'network_offer_id', 'Offer Target URL','Link Type']]
rxpcontent = rxpcontent.loc[(rxpcontent['Reporting Content ID'].isna() == False) & (rxpcontent['Reporting Content ID']!= ''),]
rxpcontent_sheet.set_dataframe(rxpcontent, start='A1')


# Update Offer Status in SMS Jump Page Content Review Google Sheet 

In [10]:
"""
smsjumppage = gc.open_by_url('https://docs.google.com/spreadsheets/d/1e8p1LMr4-StJWTF0BK0cwQzSwNLsVSFeRUD1h5pMwfA/edit#gid=1132279237')
smsjumppage_sheet = smsjumppage.worksheet('title','offers-cf')
smsjumppage_df = smsjumppage_sheet.get_as_df()
smsjumppage_df['sid'] = smsjumppage_df['sid'].astype('str').str.split('.',expand = True)[0]
smsjumppage_df1 = smsjumppage_df.merge(offer_sms[['Hitpath Offer ID','Status']], how = 'left',  left_on = 'sid',right_on ='Hitpath Offer ID',copy = False )

smsjumppage_df1['Status'] = smsjumppage_df1['Status'].fillna('')
offer_status_list_jumppage  = ['Offer Status'] +  smsjumppage_df1['Status'].tolist()
smsjumppage_sheet.update_col(2, offer_status_list_jumppage)
"""

"\nsmsjumppage = gc.open_by_url('https://docs.google.com/spreadsheets/d/1e8p1LMr4-StJWTF0BK0cwQzSwNLsVSFeRUD1h5pMwfA/edit#gid=1132279237')\nsmsjumppage_sheet = smsjumppage.worksheet('title','offers-cf')\nsmsjumppage_df = smsjumppage_sheet.get_as_df()\nsmsjumppage_df['sid'] = smsjumppage_df['sid'].astype('str').str.split('.',expand = True)[0]\nsmsjumppage_df1 = smsjumppage_df.merge(offer_sms[['Hitpath Offer ID','Status']], how = 'left',  left_on = 'sid',right_on ='Hitpath Offer ID',copy = False )\n\nsmsjumppage_df1['Status'] = smsjumppage_df1['Status'].fillna('')\noffer_status_list_jumppage  = ['Offer Status'] +  smsjumppage_df1['Status'].tolist()\nsmsjumppage_sheet.update_col(2, offer_status_list_jumppage)\n"

# Upcoming Schedule

In [11]:
#Get Code type from publisher configuration sheet
get_info= rxmgref.worksheet('title','Publisher Configurations').get_as_df()
get_info = get_info[get_info['PUBID']!=''][['PUBID','PRIMARY SC']].drop_duplicates()
get_info['PRIMARY SC'] = get_info['PRIMARY SC'].str.replace(r'\s*\[.*?\]\s*', '', regex=True)
get_info.replace('', pd.NA, inplace=True)
get_info=get_info.dropna()
get_info["Code Type"] = "Shortcode"
get_info.loc[get_info['PRIMARY SC'].astype(str).str.replace('.0','').str.len() ==11, 'Code Type'] = 'Long Code'
get_info.loc[
    (get_info['PRIMARY SC'].astype(str).str.replace('.0','').str.len() == 11) &
    (get_info['PRIMARY SC'].astype(str).str.startswith('183')),
    'Code Type'
] = 'Toll Free'
get_info.drop(columns=['PRIMARY SC'], inplace=True)

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


In [12]:
snakes = infrastructure.get_mamba()
snakes['PUBID'] = snakes['Dataset'].str.split('_',expand = True)[2].astype(int)
All_schedule = snakes.merge(publisher[['PUBID','Sub Vertical','DMA']], how = 'left', copy = False)
All_schedule = pd.merge(All_schedule, get_info, on='PUBID', how='left')
All_merge = All_schedule[(All_schedule['Date'].dt.date >= dt.date.today()) ]
All_merge = All_merge.sort_values(['Date','DMA','Sub Vertical','PUBID','Shortcode','Drop'])
offer_sms['Hitpath Offer ID'] = offer_sms['Hitpath Offer ID'].astype('str').str.split('.',expand = True)[0]
All_merge['Hitpath Offer ID'] = All_merge['Hitpath Offer ID'].astype('str').str.split('[v|.]',expand = True)[0]
All_merge_copy = All_merge.copy()
upcoming_schedule_mamba = All_merge.merge(offer_sms[['Hitpath Offer ID','Scheduling Name','Offer Name','RX Rep','Advertiser']], how = 'left', left_on = 'Hitpath Offer ID', right_on = 'Hitpath Offer ID',copy = False)
upcoming_schedule_mamba.rename(columns={'RX Rep' : 'AM'}, inplace = True)
upcoming_schedule_mamba['Offer Link'] = upcoming_schedule_mamba['Hitpath Offer ID'].apply(lambda x: f"https://www.sjejhhhe.com/rd/r.php?sid={x}&pub=460654&c1=test")
upcoming_schedule_mamba['Jump Page Link'] = upcoming_schedule_mamba['Hitpath Offer ID'].apply(lambda x: f"https://cf.freedomlender.co/?sid={x}&pub=460654&c1=test")
upcoming_schedule_mamba = upcoming_schedule_mamba[['Date','DMA','Dataset','AM','Hitpath Offer ID','Scheduling Name','Advertiser','Offer','Sub Vertical','PUBID','Drop', 'Time', 'Segment ', 'Send Strategy',
       'Creative', 'Job Name','Offer Link','Jump Page Link',"Code Type"]]

#upcoming_schedule_mamba.to_csv('SMS_upcoming_schedule.csv',index= False)

# last 60 days schedule 
All_merge_60 = All_schedule[(All_schedule['Date'] > today - dt.timedelta(days = 60)) ]
All_merge_60 = All_merge_60.sort_values(['Date','DMA','Sub Vertical','PUBID','Shortcode','Drop'], ascending = False)
All_merge_60 = All_merge_60.merge(offer_sms[['Hitpath Offer ID','Vertical']], how = 'left', left_on = 'Hitpath Offer ID', right_on = 'Hitpath Offer ID',copy = False)
upcoming_schedule_mamba_60 = All_merge_60[['Date','DMA','Sub Vertical','PUBID','Dataset','Drop', 'Time', 'Segment ', 'Send Strategy', 'Offer',
       'Creative', 'Job Name',"Code Type"]]
upcoming_schedule_mamba_60 = upcoming_schedule_mamba_60[upcoming_schedule_mamba_60['Segment '].str.split('_').str[:2].str.join('_') == upcoming_schedule_mamba_60['Dataset'].str[:-7]]
filename = local_folder+'SMS_upcoming_schedule.xlsx'
writer=pd.ExcelWriter(filename, engine='xlsxwriter')
workbook=writer.book
worksheet = workbook.add_worksheet('SMS Upcoming Schedule')
worksheet2 = workbook.add_worksheet('Last 60 Days Schedule')

#add offers list
temp = offerwall[['Hitpath Offer ID','Offer List' ]]
temp['Offer List'] = temp['Offer List'].str.replace("[","").str.replace("]","").str.split(",")
offerwall_match = temp.explode('Offer List')
offerwall_match['Offer List'] = offerwall_match['Offer List'].str.strip()
upcoming_schedule_offers_list1 = All_merge.merge(offerwall_match, on = 'Hitpath Offer ID',how = 'left')
upcoming_schedule_offers_list1['Offer Wall'] = np.nan
upcoming_schedule_offers_list1.loc[upcoming_schedule_offers_list1['Offer List'].isna() == False, 'Offer Wall'] = upcoming_schedule_offers_list1[ 'Hitpath Offer ID']
upcoming_schedule_offers_list1.loc[upcoming_schedule_offers_list1['Offer List'].isna() == False, 'Hitpath Offer ID'] = upcoming_schedule_offers_list1['Offer List']
upcoming_schedule_offers_list2 = upcoming_schedule_offers_list1.merge(offer_sms[['Hitpath Offer ID','Scheduling Name','Offer Name','RX Rep','Advertiser']], how = 'left', left_on = 'Hitpath Offer ID', right_on = 'Hitpath Offer ID',copy = False)
upcoming_schedule_offers_list2.rename(columns={'RX Rep' : 'AM'}, inplace = True)
upcoming_schedule_offers_list2['Offer Link'] = upcoming_schedule_offers_list2['Hitpath Offer ID'].apply(lambda x: f"https://www.sjejhhhe.com/rd/r.php?sid={x}&pub=460654&c1=test")
upcoming_schedule_offers_list2['Jump Page Link'] = upcoming_schedule_offers_list2['Hitpath Offer ID'].apply(lambda x: f"https://cf.freedomlender.co/?sid={x}&pub=460654&c1=test")
upcoming_schedule_offers_list = upcoming_schedule_offers_list2[['Advertiser','Hitpath Offer ID','Offer Wall','Scheduling Name','Offer','DMA','Dataset','AM','Sub Vertical','PUBID','Drop','Date','Time', 'Segment ', 'Send Strategy',
       'Creative', 'Job Name','Offer Link','Jump Page Link']]
#upcoming_schedule_offers_list = All_merge.merge(offer_sms[['Hitpath Offer ID','Scheduling Name','RX Rep','Advertiser','Offer Name']], how = 'left', left_on = 'Hitpath Offer ID', right_on = 'Hitpath Offer ID',copy = False)
#upcoming_schedule_offers_list.rename(columns={'RX Rep' : 'AM'}, inplace = True)
#upcoming_schedule_offers_list = upcoming_schedule_offers_list[['Hitpath Offer ID','Advertiser','Offer Name','AM','Date']]
upcoming_schedule_offers_list = upcoming_schedule_offers_list.sort_values(by=['Advertiser','Scheduling Name','Date','Offer Wall'], ascending = True)

#add new worksheet
worksheet3 = workbook.add_worksheet('SMS Upcoming Schedule - Offers')

worksheet.set_column(0,11,15)
worksheet2.set_column(0,11,15)
worksheet3.set_column(0,11,15)

upcoming_schedule_mamba.to_excel(writer, sheet_name='SMS Upcoming Schedule', startrow=0, startcol=0, index=False)
upcoming_schedule_mamba_60.to_excel(writer, sheet_name='Last 60 Days Schedule', startrow=0, startcol=0, index=False)
upcoming_schedule_offers_list.to_excel(writer, sheet_name='SMS Upcoming Schedule - Offers', startrow=0, startcol=0, index=False)
workbook.close()

D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\6821820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snakes['PUBID'] = snakes['Dataset'].str.split('_',expand = True)[2].astype(int)
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\6821820.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

# Loan Offer Limitation Check

In [13]:
All_merge_60['Date'] = pd.to_datetime(All_merge_60['Date'])
# Get today's date
today_date = datetime.now()
# Calculate the start and end of the current week
current_week_start = today_date - timedelta(days=today_date.weekday())
current_week_end = current_week_start + timedelta(days=6)
# Calculate the start and end of the next week
next_week_start = current_week_end + timedelta(days=1)
next_week_end = next_week_start + timedelta(days=6)
# Filter data for current week
current_week_data = All_merge_60[
    (All_merge_60['Drop'] == 'Drop 1') &
    (All_merge_60['Send Strategy'] == 'P') &
    (All_merge_60['Vertical'].str.contains('Loan')) &
    (All_merge_60['Date'] >= current_week_start) & (All_merge_60['Date'] <= current_week_end)
][['shortcode_DP.SV', 'Shortcode', 'Hitpath Offer ID', 'DMA']]
# Filter data for next week
next_week_data = All_merge_60[
    (All_merge_60['Drop'] == 'Drop 1') &
    (All_merge_60['Send Strategy'] == 'P') &
    (All_merge_60['Vertical'].str.contains('Loan')) &
    (All_merge_60['Date'] >= next_week_start) & (All_merge_60['Date'] <= next_week_end)
][['shortcode_DP.SV', 'Shortcode', 'Hitpath Offer ID', 'DMA']]
# Group by 'shortcode_DP.SV' and count the number of loan offers for current week
current_week_data.drop_duplicates(subset=['shortcode_DP.SV', 'Hitpath Offer ID'], keep='first', inplace=True)
next_week_data.drop_duplicates(subset=['shortcode_DP.SV', 'Hitpath Offer ID'], keep='first', inplace=True)

current_week_counts = current_week_data.groupby(['shortcode_DP.SV', 'Shortcode']).size().reset_index(name='Current Week Count')
# Group by 'shortcode_DP.SV' and count the number of loan offers for next week
next_week_counts = next_week_data.groupby(['shortcode_DP.SV', 'Shortcode']).size().reset_index(name='Next Week Count')
#merge
total_counts = pd.merge(current_week_counts, next_week_counts, on=['shortcode_DP.SV', 'Shortcode'], how='outer').fillna(0)
# Dictionary of weekly loan limits
loan_limit_weekly = {
    "FLC": 3,
    "SVT": 3,
    "DSS": 3,
    "HZB": 3,
    "MBC": 2,
    "UAA": 2,
    "MFA": 2,
    "N3G": 3
}
# Check limits
total_counts['Current Week Exceeds Limit'] = total_counts.apply(
    lambda row: row['Current Week Count'] > loan_limit_weekly.get(row['Shortcode'], float('inf')), axis=1)
total_counts['Next Week Exceeds Limit'] = total_counts.apply(
    lambda row: row['Next Week Count'] > loan_limit_weekly.get(row['Shortcode'], float('inf')), axis=1)
# Merge total_counts with filtered current and next week data to get relevant Hitpath Offer ID
results_current = pd.merge(total_counts[total_counts['Current Week Exceeds Limit']], current_week_data, on=['shortcode_DP.SV', 'Shortcode'], how='left')
results_next = pd.merge(total_counts[total_counts['Next Week Exceeds Limit']], next_week_data, on=['shortcode_DP.SV', 'Shortcode'], how='left')
# Print the Hitpath Offer IDs for rows where limits are exceeded, grouped by 'shortcode_DP.SV'
current_ids = results_current.groupby(['shortcode_DP.SV', 'DMA'])['Hitpath Offer ID'].apply(list).to_dict()
next_ids = results_next.groupby(['shortcode_DP.SV', 'DMA'])['Hitpath Offer ID'].apply(list).to_dict()

In [14]:
#current_ids

In [15]:
#current_week_data

In [16]:
upcomingschedule = mamba.worksheet('title','SMS Upcoming Schedule')
upcomingschedule.clear()
upcomingschedule.set_dataframe(upcoming_schedule_mamba,(1,1))
upcomingschedule.cell('A1').set_text_format('bold', True)
upcomingschedule.cell('B1').set_text_format('bold', True)
upcomingschedule.cell('C1').set_text_format('bold', True)
upcomingschedule.cell('D1').set_text_format('bold', True)
upcomingschedule.cell('E1').set_text_format('bold', True)
upcomingschedule.cell('F1').set_text_format('bold', True)
upcomingschedule.cell('G1').set_text_format('bold', True)
upcomingschedule.cell('H1').set_text_format('bold', True)
upcomingschedule.cell('I1').set_text_format('bold', True)
upcomingschedule.cell('J1').set_text_format('bold', True)
upcomingschedule.cell('K1').set_text_format('bold', True)
upcomingschedule.cell('L1').set_text_format('bold', True)
upcomingschedule.cell('M1').set_text_format('bold', True)
upcomingschedule.cell('N1').set_text_format('bold', True)
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.72, 'green': 0.64, 'blue': 0.81}}, ['Personal Loan'])
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.95, 'green': 0.75, 'blue': 0.41}}, ['Sweepstakes'])
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.64, 'green': 0.72, 'blue': 0.81}}, ['Rent To Own'])
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.63, 'green': 0.85, 'blue': 0.73}}, ['Credit Card'])
upcomingschedule.add_conditional_formatting('C1', 'C1000', 'TEXT_CONTAINS', {'backgroundColor':{'red': 0.95, 'green': 0.94, 'blue': 0.64}}, ['Financial'])

# Email offer Sync up in SMS Offer Sheet

In [17]:
sync =  pd.read_csv(filepath.output_folder+'offer sync history.csv')

In [18]:
same_offer_from_email = offer_sms[['Hitpath Offer ID','Scheduling Name','Email Hitpath Offer ID','Status','$ Payout','Payout Change','Paused Pubs','Special Pub Payouts','Day Restrictions','Cap', 'Budget']].merge(offer_email_raw_data[['Hitpath Offer ID from email','Scheduling Name from email','Operational Status from email','Payout from email','Paused PubIDs from email','Day Restrictions from email','Drop Time Restrictions from email','Budget from email', 'Cap from email']],left_on='Email Hitpath Offer ID', right_on = 'Hitpath Offer ID from email', how = 'inner')
same_offer_from_email['Hitpath Offer ID'] = same_offer_from_email['Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]
#offer_sms['Email Hitpath Offer ID'] = offer_sms['Email Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]
email_list = offer_sms['Email Hitpath Offer ID'].unique().tolist()
email_list = [x for x in email_list if x !='nan']
sms_live_pubid = publisher['PUBID'].astype(str).unique().tolist()
# special payout for 9287 and 7878 
offer9287['PUBID'] = offer9287['PUBID'].astype(str).str.split(".",expand = True)[0]
sp9287 = offer9287.loc[(offer9287['STATUS'] == 'Live') & (offer9287['PUBID'].astype(str).isin(sms_live_pubid)) ,][['PUBID','NEW RATE']].to_string(index=False,header =False)
offer7878['PUBID'] = offer7878['PUBID'].astype(str).str.split(".",expand = True)[0]
sp7878 = offer7878.loc[(offer7878['STATUS'] == 'Live') & (offer7878['PUBID'].astype(str).isin(sms_live_pubid)) ,][['PUBID','NEW RATE']].to_string(index=False,header = False)
if "Empty DataFrame" in sp7878:
    sp7878 = ''
offer = same_offer_from_email[same_offer_from_email['Email Hitpath Offer ID'].isnull() == False]

# speical payout 
offer['Special Payout from Email'] = ''
offer['Special Pub Payouts'] = offer['Special Pub Payouts'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer.loc[offer['Email Hitpath Offer ID'] == 9287,'Special Payout from Email' ] = sp9287
offer.loc[offer['Email Hitpath Offer ID'] == 7878,'Special Payout from Email' ] = sp7878
offer['Special Payout Alert'] = ''
offer.loc[(offer['Special Pub Payouts']!=offer['Special Payout from Email']) , 'Special Payout Alert'] = 'Please update Special Payout'

offer['Paused Alert'] =  ''
offer.loc[(offer['Status'].str.lower().str.contains('set up|paused|pitch|pause',na = False) == False) & (offer['Operational Status from email'].str.contains('Paused') ),'Paused Alert'] = 'Please Pause offer in SMS'

### newly added
offer['Hitpath Offer ID'] = offer['Hitpath Offer ID'].astype('int64')
offer = pd.merge(offer, sync[['Hitpath Offer ID', 'Paused Alert Start Date']], on='Hitpath Offer ID', how='left')
offer['Paused Alert Start Date'] = offer['Paused Alert Start Date'].fillna('')

def apply_alert_logic(row):
    #current_time = pd.Timestamp.now() + pd.Timedelta(days=91)
    current_time = pd.Timestamp.now()
    first_time = len(row['Paused Alert Start Date']) == 0
    if first_time:
        date_str = pd.Timestamp('today').strftime('%Y-%m-%d')
        duration_months = 0
    else:
        date_str = row['Paused Alert Start Date']
        added_date = datetime.strptime(date_str, '%Y-%m-%d') 
        duration_months = (current_time - pd.Timestamp(added_date)).days / 30
     
    # Check if 'Paused Alert' is empty as a proxy for determining if it's the first time
    has_alert = len(row['Paused Alert']) > 0
    if has_alert and duration_months < 3:
        return pd.Series([row['Paused Alert'], date_str], index=['Paused Alert', 'Paused Alert Start Date'])
        
    if duration_months >= 3:
        return pd.Series(['', date_str], index=['Paused Alert', 'Paused Alert Start Date'])
    
    return pd.Series([row['Paused Alert'], row['Paused Alert Start Date']], index=['Paused Alert', 'Paused Alert Start Date'])

# Apply the function across the DataFrame
offer[['Paused Alert', 'Paused Alert Start Date']] = offer.apply(apply_alert_logic, axis=1)
###end

offer['Payout Alert'] = ''
offer['SMS_payout_rate'] = offer['$ Payout'].str.extract(r'([\d\,.]+)')
offer['SMS_payout_rate'] = offer['SMS_payout_rate'].str.replace(",","").astype(float)
offer['SMS_new_payout_rate'] = offer['Payout Change'].str.extract(r'([\d\,.]+)')
offer['SMS_new_payout_rate'] = offer['SMS_new_payout_rate'].str.replace(",","").astype(float)
offer['Email_payout_rate'] = offer['Payout from email'].str.extract(r'([\d\,.]+)')
offer['Email_payout_rate'] = offer['Email_payout_rate'].str.replace(",","").astype(float)
offer['Payout Alert'] = ''
offer.loc[(offer['SMS_payout_rate']!=offer['Email_payout_rate']) & (offer['SMS_new_payout_rate']!=offer['Email_payout_rate'])&(offer['$ Payout']!=offer['Payout from email']), 'Payout Alert'] = 'Please update the payout'

offer['Paused Pubids list'] = offer['Paused PubIDs from email'].str.split('\n|\t|,|"| ')
offer['SMS current Paused Pubids list'] = offer['Paused Pubs'].str.split('\n|\t|,|"| ')
offer['SMS current Paused Pubids list'] = offer['SMS current Paused Pubids list'].apply(lambda x: sorted(x) if isinstance(x, (list, tuple))  else list())
offer['Update Needed in SMS: Paused Pubids'] = offer['Paused Pubids list'].apply(lambda x: sorted(list(set(list(x)) & set(sms_live_pubid))) if isinstance(x, (list, tuple))  else list())
offer['Update Needed in SMS: Paused Pubids'] = offer['Update Needed in SMS: Paused Pubids'].apply(lambda x: [int(i) for i in x if i != ''])
offer['SMS current Paused Pubids list'] = offer['SMS current Paused Pubids list'].apply(lambda x: [int(i) for i in x if i != ''])
offer.loc[offer['Update Needed in SMS: Paused Pubids'] == offer['SMS current Paused Pubids list'],'Update Needed in SMS: Paused Pubids'] = ''
offer['Update Needed in SMS: Paused Pubids'] = offer['Update Needed in SMS: Paused Pubids'].apply(lambda x: '' if x == list() else x) 
offer['Update Needed in SMS: Paused Pubids'] = offer['Update Needed in SMS: Paused Pubids'].apply(lambda x: '\n'.join(map(str, x)))
offer.loc[offer['SMS current Paused Pubids list'].isnull(), 'SMS current Paused Pubids list'] = ''

offer['Day Restrictions from Email'] = offer['Day Restrictions from email'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x ) + ' ' + offer['Drop Time Restrictions from email'].apply(lambda x: "" if x == 'None'  or pd.isnull(x) else x )
offer['Day Restrictions'] = offer['Day Restrictions'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer['Day Restrictions Alert'] = ''
offer.loc[offer['Day Restrictions'].str.replace(" ","").str.lower()!=offer['Day Restrictions from Email'].str.replace(" ","").str.lower(),'Day Restrictions Alert'] = 'Please update the Day Restrictions'

offer['Budget Alert'] =''
offer['Cap from email1'] = offer['Cap from email'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer['Cap1'] = offer['Cap'].apply(lambda x: "" if x == 'No Cap'  or pd.isnull(x) else x )
#offer['New Cap Budget1'] = offer['New Cap Budget'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer['Budget from email1'] = offer['Budget from email'].apply(lambda x: "" if x == 'None'  or pd.isnull(x) else x )
offer['Cap Budget1'] = offer['Budget'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
offer.loc[offer['Cap1']!=offer['Cap from email1'],'Budget Alert'] = offer['Budget Alert']+"Please update budget cap. "
offer['Budget from email2'] = offer['Budget from email1'].str.extract(r'(\d+)')
offer.loc[(offer['Budget from email1']!=offer['Cap Budget1'])&(offer['Budget from email2']!=offer['Cap Budget1']), 'Budget Alert'] = offer['Budget Alert']+"Please update budget. "
offer = offer.drop(['SMS_payout_rate','Email_payout_rate','Paused Pubids list','SMS current Paused Pubids list','SMS_new_payout_rate','Cap from email1','Cap1','Budget from email1','Cap Budget1','Budget from email2'],axis = 1)
# if offer is paused, then all alert is ''

offer.loc[(offer['Status'].str.contains('Pause')), 'Budget Alert'] = ''
offer.loc[(offer['Status'].str.contains('Pause')), 'Update Needed in SMS: Paused Pubids'] = ''
offer.loc[(offer['Status'].str.contains('Pause')), 'Paused Alert'] = ''
offer.loc[(offer['Status'].str.contains('Pause')), 'Payout Alert'] = ''
offer.loc[(offer['Status'].str.contains('Pause')), 'Day Restrictions Alert'] = ''
offer.loc[(offer['Status'].str.contains('Pause')), 'Special Payout Alert'] = ''
#offer[(offer['SMS current Paused Pubids list'] !=offer['Update Needed in SMS: Paused Pubids'])]
#export into local file
offer.to_csv(local_folder+'offer sync history.csv', index = False)

# communicte with Daniel to uniform the time format 
offer_upload = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Y0NjJBQBSXhTangk61PrOlKJBNZat9mmZGanAPuLma4/edit#gid=0')
offer_upload_wks  = offer_upload.sheet1
headers = offer_upload_wks.get_row(1)
offer_upload_wks.clear(start='A2')
offer_upload_wks.update_row(1, headers)
offer_upload_wks.set_dataframe(offer,(2,1))

# EDIT EMAIL MESSAGE: 
need_update_offer_list = offer.loc[((offer['Update Needed in SMS: Paused Pubids']!= '') | (offer['Paused Alert']!= '') | (offer['Payout Alert']!= '') |  (offer['Day Restrictions Alert']!= '') | (offer['Budget Alert']!= '') | (offer['Special Payout Alert']!='')), ]['Hitpath Offer ID' ].unique().tolist()
offer_list_details = pd.merge(offer, offer_sms, on = 'Hitpath Offer ID', how = 'inner')
filtered_offer_list_details = offer_list_details[offer_list_details['Hitpath Offer ID'].isin(need_update_offer_list)]

combined_list_values = []

#iterate over each row
for index, row in filtered_offer_list_details.iterrows():
    # Append the 'Hitpath Offer ID' to the combined list
    combined_list_values.append(row['Hitpath Offer ID'])
    # Append the corresponding values from OLD_RXRep and OLD_Offername to the combined list
    combined_list_values.append(row['RX Rep'])
#     combined_list_values.append(row['Offer Name'])
    combined_list_values.append('\n')

#OLD_RXRep = filtered_offer_list_details['RX Rep'].tolist()
#OLD_Offername = filtered_offer_list_details['Offer Name'].tolist()
#need_update_offer_list += OLD_RXRep + OLD_Offername
need_update_offer_list_str = ','.join(map(str, combined_list_values))
need_update_offer_list_str = need_update_offer_list_str.replace(',\n', '\n')

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\3920784215.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer['Special Payout from Email'] = ''
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\3920784215.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer['Special Pub Payouts'] = offer['Special Pub Payouts'].apply(lambda x: "" if str(x).lower() == 'none'  or pd.isnull(x) else x )
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\3920784215.py:21: SettingWithCopyWarning: 
A value is t

In [19]:
need_update_offer_list

['6763', '6322', '12588', '12587', '12306', '12305']

In [20]:
need_update_offer_list_str 

'6763,Marley Smith\n,6322,Marley Smith\n,12588,Kelly baker\n,12587,Kelly baker\n,12306,Jennifer Sporleder\n,12305,Jennifer Sporleder\n'

In [21]:

if len(need_update_offer_list) >0: 
    message = "Hi AM Team, we found the information discrepancy between SMS Offer Sheet and Email Offer Management Smartsheet for the following email offers and please update the information in the SMS Offer Sheet. Thank you!\nHere's the list: \n" + need_update_offer_list_str

    receipient_list = ['nina@rxmg.com']
    #receipient_list =  ['offernotices@rxmg.com','lili@rxmg.com','nina@rxmg.com']
    for i in receipient_list:
        offer_upload.share(i, role='commenter', type='user', emailMessage=message)


# Swap Report 

In [22]:
def get_day_list(date_range):
    days_of_week = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']
    try: 
        start_day, end_day = date_range.split('-')

        start_index = days_of_week.index(start_day.strip())
        end_index = days_of_week.index(end_day.strip())

        if start_index <= end_index:
            selected_days = days_of_week[start_index:end_index + 1]
        else:
            selected_days = days_of_week[start_index:] + days_of_week[:end_index + 1]

        return ','.join(selected_days)
    except:
        return ','.join(days_of_week)

In [23]:
 
offer_sms['Day Restrictions'] = offer_sms['Day Restrictions'].str.upper()
offer_sms['Day Restrictions'] = offer_sms['Day Restrictions'].str.replace(' ','')
offer_sms['Day Restrictions'] = offer_sms['Day Restrictions'].str.replace('M-F','MONDAY-FRIDAY')
upcoming_schedule_offers_list1['Hitpath Offer ID'] = upcoming_schedule_offers_list1['Hitpath Offer ID'].astype('str').str.split('.',expand = True)[0]
offer_sms['Hitpath Offer ID'] = offer_sms['Hitpath Offer ID'].astype('str').str.split('.',expand = True)[0]

All_merge.rename(columns={'Hitpath Offer ID':'Hitpath ID'}, inplace = True)
offer_sms.rename(columns={'Hitpath Offer ID':'Hitpath ID'}, inplace = True)
upcoming_schedule_offers_list1.rename(columns={'Hitpath Offer ID':'Hitpath ID'}, inplace = True)

alert_report = upcoming_schedule_offers_list1.merge(offer_sms[['Hitpath ID','Email Hitpath Offer ID','RX Rep','Scheduling Name','Offer Name','Advertiser','Day Restrictions','Status','Paused Pubs']], left_on ='Hitpath ID',right_on = 'Hitpath ID', how = 'left') 
offer_email = offer_email[offer_email['Hitpath Offer ID'].isna() == False]
alert_report = alert_report.merge(offer_email[['Hitpath Offer ID','Operational Status']],left_on ='Email Hitpath Offer ID' ,right_on ='Hitpath Offer ID', how = 'left',copy= False)

alert_report['Alert'] = ''
alert_report.loc[alert_report['Creative'].isna(),'Alert'] = 'No Creative'
alert_report['Weekday'] = pd.to_datetime(alert_report['Date']).dt.day_name()
alert_report.loc[alert_report['Creative'].isna(),'Alert'] = 'No Creative'
alert_report.loc[alert_report['Status'].str.lower().str.contains('set up|paused|pitch|archive',na = False), 'Alert' ] = 'Not Live Offer'
alert_report.loc[(alert_report['Status'].str.lower().str.contains('set up|paused|pitch|archive',na = False)) & (alert_report['Offer Wall'].isna() == False), 'Alert' ] = 'Not Live Offer in Offer Wall'
for i in holiday_dict: 
    alert_report.loc[(alert_report['Date'] == i) & (alert_report['Hitpath ID'].isin(holiday_dict[i])), 'Alert' ] = 'Holiday Restriction'
    alert_report.loc[(alert_report['Date'] == i) & (alert_report['Hitpath ID'].isin(holiday_dict[i])) &(alert_report['Offer Wall'].isna() == False) , 'Alert' ] = 'Holiday Restriction in Offer Wall'
    
# alert_report.loc[(alert_report['Operational Status'].str.lower().str.contains('paused',na = False)),  'Alert'  ] = 'Not Live Offer in Email'
alert_report['Day Allow to send'] = alert_report['Day Restrictions'].apply(lambda x: get_day_list(x))
alert_report['DayIsAllowed'] = alert_report.apply(lambda x: x['Weekday'].upper() in x['Day Allow to send'], axis=1)
alert_report.loc[(alert_report['DayIsAllowed']==False  ) ,'Alert'] = 'Day Restriction'
alert_report.loc[(alert_report['DayIsAllowed']==False  )& (alert_report['Offer Wall'].isna() == False) ,'Alert'] = 'Day Restriction in Offer Wall'
alert_report['Paused Pubs'] = alert_report['Paused Pubs'].astype('str')
alert_report['PUBID'] = alert_report['PUBID'].astype('str')
alert_report['Paused Alert'] = alert_report.apply(lambda x:x['PUBID'] in x['Paused Pubs'], axis=1)
alert_report.loc[alert_report['Paused Alert']== True,'Alert'] = 'Paused Pubid'
alert_report.loc[(alert_report['Paused Alert']== True)& (alert_report['Offer Wall'].isna() == False) ,'Alert'] = 'Paused Pubid in Offer Wall'
alert_report['Responsible DMA'] = alert_report['DMA']
alert_report['AM'] = alert_report['RX Rep']
alert_report.loc[alert_report['Send Strategy'] == 'PT','Responsible DMA' ] = 'Nathan'
alert_report = alert_report[alert_report['Alert'] != ''][['Responsible DMA','Alert','AM','Hitpath ID','Offer Wall','Offer Name','Advertiser','Day Restrictions','Status','Date','Sub Vertical','PUBID','Dataset','Drop', 'Time', 'Segment ', 'Send Strategy', 'Offer',
       'Creative']]
# alert from email offer 

#alert_report.to_excel(filename,sheet_name='Production Drops',  index = False)


# AR Swap Report 

In [24]:
# arschedule

In [25]:
arschedule['Added?'] = arschedule['Added?'].astype(str)
arschedule.dtypes

DP.DS                           object
Date Started                    object
Date Ended                      object
Added?                          object
Peer Check                      object
Send Notification to Bose       object
Day                             object
Shortcode                       object
Account Owner                   object
Message                         object
Offer ID                       float64
OWID                           float64
Offer Wall C1 Identifier        object
New Offer Name                  object
SS Offer Link (temp column)     object
New Offer Link                  object
DP.DS or DP.sV                  object
PUBID                           object
DMA                             object
dtype: object

In [26]:
arschedule['Offer ID'] = arschedule['Offer ID'].astype('str').str.split('.',expand = True)[0]
arschedule.loc[arschedule['Offer ID']=='', 'Offer ID'] = arschedule['Offer Wall C1 Identifier']
arschedule1 = arschedule.merge(offerwall_match, left_on = 'Offer ID', right_on = 'Hitpath Offer ID',how = 'left')
arschedule1['Offer Wall'] = np.nan
arschedule1.loc[arschedule1['Offer List'].isna() == False, 'Offer Wall'] = arschedule1[ 'Offer ID']
arschedule1.loc[arschedule1['Offer List'].isna() == False, 'Offer ID'] = arschedule1['Offer List']
arschedule_swap = arschedule1.merge(offer_sms[['Hitpath ID','RX Rep','Scheduling Name','Offer Name','Advertiser','Day Restrictions','Status','Paused Pubs']], left_on ='Offer ID',right_on = 'Hitpath ID', how = 'left') 
arschedule_swap['Alert']  = ''
arschedule_swap.loc[arschedule_swap['Status'].str.lower().str.contains('set up|paused|pitch|archive',na = False), 'Alert' ] = 'Not Live Offer'
arschedule_swap.loc[(arschedule_swap['Status'].str.lower().str.contains('set up|paused|pitch|archive',na = False)) & (arschedule_swap['Offer Wall'].isna() == False), 'Alert' ] = 'Not Live Offer in Offer Wall'
arschedule_swap.loc[ (arschedule_swap['Day Restrictions'].isna() == False),'Alert'] = 'Day Restriction'
arschedule_swap.loc[ (arschedule_swap['Day Restrictions'].isna() == False)& (arschedule_swap['Offer Wall'].isna() == False) ,'Alert'] = 'Day Restriction in Offer Wall'
arschedule_swap['Paused Pubs'] = arschedule_swap['Paused Pubs'].astype('str')
arschedule_swap['PUBID'] = arschedule_swap['PUBID'].astype('str')
arschedule_swap['Paused Alert'] = arschedule_swap.apply(lambda x:x['PUBID'] in x['Paused Pubs'], axis=1)
arschedule_swap.loc[arschedule_swap['Paused Alert']== True,'Alert'] = 'Paused Pubid'
arschedule_swap.loc[(arschedule_swap['Paused Alert']== True) &  (arschedule_swap['Offer Wall'].isna() == False),'Alert'] = 'Paused Pubid in Offer Wall'
for i in holiday_dict: 
    arschedule_swap.loc[(( i-datetime.today()).days>0) & (( i-datetime.today()).days<=7) & (arschedule_swap['Hitpath ID'].isin(holiday_dict[i])), 'Alert' ] = 'Upcoming Holiday Restriction'
    arschedule_swap.loc[ (( i-datetime.today()).days>0) & (( i-datetime.today()).days<=7)&(arschedule_swap['Hitpath ID'].isin(holiday_dict[i])) &(arschedule_swap['Offer Wall'].isna() == False) , 'Alert' ] = 'Upcoming Holiday Restriction in Offer Wall'
arschedule['Date Started'] = pd.to_datetime(arschedule['Date Started'])
arschedule = arschedule.merge(offer_sms[['Hitpath ID','RX Rep','Scheduling Name','Offer Name','Advertiser','Day Restrictions','Status','Paused Pubs']], left_on ='Offer ID',right_on = 'Hitpath ID', how = 'left') 
arschedule.rename(columns = {'RX Rep':'AM','Responsible DMA':'DMA','New Offer Name': 'AR offer Name'},inplace = True)
arschedule_swap.rename(columns = {'RX Rep':'AM','Responsible DMA':'DMA','New Offer Name': 'AR offer Name'}, inplace = True)
recent_update_ar = arschedule.loc[(arschedule['Date Started']>= (datetime.now() - timedelta(days=30))) & (arschedule['Peer Check']=="FALSE")][['AR offer Name','DMA','Date Started',"Peer Check",'AM','PUBID','Offer ID','Offer Name','Advertiser']].sort_values(['DMA','AR offer Name'])
ar_need_to_add = arschedule.loc[arschedule['Added?']!='True'][['AR offer Name','DMA','Date Started',"Peer Check",'AM','PUBID','Offer ID','Offer Name','Advertiser']].sort_values('AR offer Name')
current_ar = arschedule[['DMA','AM','AR offer Name','PUBID','Offer ID','Offer Name','Advertiser']].sort_values('AR offer Name')
current_ar_offer_list = arschedule_swap[['Offer ID','Offer Wall','Offer Name','Advertiser','AM','DMA','AR offer Name','PUBID']].sort_values(['Advertiser','Offer ID'])
arschedule_swap_alert = arschedule_swap[arschedule_swap['Alert'] != ''][['DMA','Alert','Offer ID','Offer Wall','AM','AR offer Name','PUBID','Offer Name','Advertiser','Day Restrictions','Status']].sort_values('AR offer Name')


D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\2480485340.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arschedule['Date Started'] = pd.to_datetime(arschedule['Date Started'])


In [27]:
# Create an ExcelWriter object
filename = local_folder+'AR_schedule.xlsx'
writer=pd.ExcelWriter(filename, engine='xlsxwriter')
workbook=writer.book
worksheet = workbook.add_worksheet('All AR Drops')
worksheet1 = workbook.add_worksheet('AR Drops Awaiting Add')
worksheet2 = workbook.add_worksheet('AR Drops Awaiting Peer Check')
worksheet3 = workbook.add_worksheet('All AR Drops - Offer List')
worksheet.set_column(0,10,15)
worksheet1.set_column(0,10,15)
worksheet2.set_column(0,10,15)
worksheet3.set_column(0,10,15)
current_ar.to_excel(writer, sheet_name='All AR Drops', startrow=0, startcol=0, index=False)
ar_need_to_add.to_excel(writer, sheet_name='AR Drops Awaiting Add', startrow=0, startcol=0, index=False)
recent_update_ar.to_excel(writer, sheet_name='AR Drops Awaiting Peer Check', startrow=0, startcol=0, index=False)
current_ar_offer_list.to_excel(writer, sheet_name='All AR Drops - Offer List', startrow=0, startcol=0, index=False)
workbook.close()

# Underperforming Swap Report

In [28]:

# Underperfoming Swap Report 
# 4 day before today 

four_days_ago = today - timedelta(days=4)
lexi =  pd.read_csv(filepath.output_folder+'SS_LC_merged_data.csv')
lexi1 = lexi.loc[lexi['Send Strategy']=='P']
# only take this month data 
lexi1['Date'] = pd.to_datetime(lexi1['Date'])
# last drop 
gpby_last_drop = lexi1.sort_values(by = 'Date', ascending= False).groupby(['shortcode_DP.SV','Hitpath_Offer_ID']).first()
gpby_last_drop = gpby_last_drop.reset_index()
gpby_last_drop1 = gpby_last_drop.groupby(['shortcode_DP.SV','Hitpath_Offer_ID','Date']).agg({'Revenue':'sum','Delivered':'sum'})
gpby_last_drop1['eCPM'] = gpby_last_drop1['Revenue'] / gpby_last_drop1['Delivered'] * 1000

gpby_last_drop1 = gpby_last_drop1.reset_index()
gpby_last_drop1.rename(columns={'Date':'Last_Drop_Date','Revenue':'Last_Drop_Revenue','Delivered':'Last_Drop_Delivered','eCPM':'Last_Drop_eCPM'}, inplace=True)
All_merge = pd.merge(All_merge, offer_sms[['Hitpath ID','RX Rep','Scheduling Name','Offer Name','Advertiser']], left_on = 'Hitpath ID', right_on = 'Hitpath ID', how = 'left')
All_merge.rename(columns={'RX Rep' : 'AM'}, inplace = True)
upcoming_schedule_mamba_hit = All_merge[['Date','DMA','AM','Hitpath ID','Offer Name','Advertiser','Sub Vertical','PUBID','Dataset','Drop', 'Time', 'Segment ', 'Send Strategy', 'Offer',
       'Creative', 'Job Name','shortcode_DP.SV']]
mamba_upcoming_drop = upcoming_schedule_mamba_hit.merge(gpby_last_drop1, left_on=['shortcode_DP.SV','Hitpath ID'],right_on = ['shortcode_DP.SV','Hitpath_Offer_ID'],how='left')
underperforming_swap = mamba_upcoming_drop.loc[(mamba_upcoming_drop['Last_Drop_eCPM'] < 7) & ((mamba_upcoming_drop['Date'] - mamba_upcoming_drop['Last_Drop_Date']).dt.days <=8) & (mamba_upcoming_drop['Last_Drop_Date'] <= four_days_ago)  ,] 
underperforming_swap.rename(columns={'Hitpath ID':'Hitpath Offer ID'}, inplace = True)
underperforming_swap.drop(columns = ['Offer','Creative','Job Name','shortcode_DP.SV','Hitpath_Offer_ID','Time'], inplace = True)

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\3075404448.py:5: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  lexi =  pd.read_csv(filepath.output_folder+'SS_LC_merged_data.csv')
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\3075404448.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lexi1['Date'] = pd.to_datetime(lexi1['Date'])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\3075404448.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

## Light Traffic

In [29]:
def light_traffic(url): 
    light_traffic_sheet = gc.open_by_url(url).worksheet('title','SMS Drop Restrictions')
    light_traffic_df = light_traffic_sheet.get_as_df()
    light_traffic_df.columns = light_traffic_df.iloc[5]
    light_traffic_df1= light_traffic_df.iloc[7:]
    df_filtered = light_traffic_df1[light_traffic_df1['LIMITED TRAFFIC (%)'].astype(bool)]
    df_filtered['LIMITED TRAFFIC (%)'] = df_filtered['LIMITED TRAFFIC (%)'].str.rstrip('%').astype(float) / 100
    df_filtered = df_filtered.reset_index(drop=True)
    return df_filtered

def get_last_seven_days_avg(df):
    # Sort by 'Hitpath Offer ID' and 'Date' in descending order
    df = df.sort_values(by=['Hitpath Offer ID', 'Date'], ascending=[True, False])
    
    date_threshold = datetime.now() - timedelta(days=14)
    
    # Filter rows where Date is within the last 14 days
    df = df[df['Date'] >= date_threshold]
    # Group by 'Hitpath Offer ID' and get the last 7 distinct dates
    result = df.groupby('Hitpath Offer ID').apply(lambda x: x.drop_duplicates(subset='Date').head(7)).reset_index(drop=True)
    
     # Filter out groups with fewer than 7 distinct dates
    valid_groups = result.groupby('Hitpath Offer ID').filter(lambda x: len(x) == 7)
    
    # Calculate the average 'Delivered' value for each valid 'Hitpath Offer ID'
    avg_delivered = valid_groups.groupby('Hitpath Offer ID')['Delivered'].mean().reset_index()
    avg_delivered.columns = ['Hitpath Offer ID', 'Average Delivered']
    
    return avg_delivered

def convert_pause_period(pause_period):
    dates = pause_period.split(' - ')
    start_date = pd.to_datetime(dates[0])
    end_date = pd.to_datetime(dates[1]) if len(dates) > 1 else start_date
    return start_date, end_date
    


light_traffic_df = light_traffic('https://docs.google.com/spreadsheets/d/127S2S6Krl7KZJzBlxKJg8L2HrZqWAoJ8XAPGEFGhOVc/edit?gid=832173927#gid=832173927')
#holiday_dict = holiday_restriction('https://docs.google.com/spreadsheets/d/1USZQ_CIpfEV-NDH8K9oVy-hy3cTh4uaD0SuksRCtEq8/edit#gid=1068286208')

light_traffic_df['HitPath ID'] = light_traffic_df['HitPath ID'].astype(str).str.split('.',expand = True)[0]
light_traffic_df.rename(columns={'HitPath ID' : 'Hitpath Offer ID'}, inplace = True)
light_traffic_df = light_traffic_df[['Scheduling Name', 'Hitpath Offer ID','Offer Name','Network', 'OVERALL PAUSE PERIOD', 'LIMITED TRAFFIC (%)']]
light_traffic_df[['Pause Start', 'Pause End']] = light_traffic_df['OVERALL PAUSE PERIOD'].apply(lambda x: pd.Series(convert_pause_period(x)))


# past record
light_traffic_past_set = lexi1[lexi1['Hitpath_Offer_ID'].isin(light_traffic_df['Hitpath Offer ID'].unique())]
light_traffic_past_set.rename(columns={'Hitpath_Offer_ID' : 'Hitpath Offer ID'}, inplace = True)




light_traffic_past_groupby = light_traffic_past_set.groupby(['Hitpath Offer ID','Date']).agg({'Delivered':'sum'}).reset_index()


last_7_drop_avg = get_last_seven_days_avg(light_traffic_past_groupby)

last_7_drop_bound = pd.merge(last_7_drop_avg, light_traffic_df[['Hitpath Offer ID', 'LIMITED TRAFFIC (%)']], on='Hitpath Offer ID')
last_7_drop_bound['Limited Traffic'] = last_7_drop_bound['LIMITED TRAFFIC (%)'] * last_7_drop_bound['Average Delivered']

# start_date = datetime.datetime.now() - timedelta(days=7)
# last_seven_offer_data = offer_data[offer_data['Date'] >= start_date]

upcoming_light_traffic = mamba_upcoming_drop[mamba_upcoming_drop['Hitpath ID'].isin(light_traffic_df['Hitpath Offer ID'].unique())]
upcoming_light_traffic.rename(columns={'Segment ' : 'Segment'}, inplace = True)
upcoming_light_traffic['Segment'] = upcoming_light_traffic['Segment'].str.replace('\n', '')

upcoming_light_traffic_offer = pd.merge(upcoming_light_traffic, light_traffic_df[['Hitpath Offer ID', 'Pause Start', 'Pause End']], left_on = 'Hitpath ID', right_on = 'Hitpath Offer ID')

filtered_upcoming_light_traffic_offer = upcoming_light_traffic_offer[(upcoming_light_traffic_offer['Date'] >= upcoming_light_traffic_offer['Pause Start']) & (upcoming_light_traffic_offer['Date'] <= upcoming_light_traffic_offer['Pause End'])]

filtered_upcoming_light_traffic_offer

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_10416\3832527760.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['LIMITED TRAFFIC (%)'] = df_filtered['LIMITED TRAFFIC (%)'].str.rstrip('%').astype(float) / 100
D:\Users\lilig\AppData\Local\Temp\1\ipyke

,Date,DMA,AM,Hitpath ID,Offer Name,Advertiser,Sub Vertical,PUBID,Dataset,Drop,...,Job Name,shortcode_DP.SV,Hitpath_Offer_ID,Last_Drop_Date,Last_Drop_Revenue,Last_Drop_Delivered,Last_Drop_eCPM,Hitpath Offer ID,Pause Start,Pause End


In [30]:
segments =  pd.read_csv(filepath.output_folder+'SMS_SC_SS_Segments.csv')
segments.rename(columns={'name' : 'Segment'}, inplace = True)

segments['last_counted'] = pd.to_datetime(segments['last_counted'], errors='coerce')
# Drop rows where 'last_counted' is NaT (empty or invalid dates)
segments = segments.dropna(subset=['last_counted'])
latest_segment_df = segments.loc[segments.groupby('Segment')['last_counted'].idxmax()]

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [31]:
filtered_upcoming_light_traffic_offer_segment = pd.merge(filtered_upcoming_light_traffic_offer, latest_segment_df[['Segment', 'count']], on='Segment', how='inner')
filtered_upcoming_light_traffic_offer_segment

,Date,DMA,AM,Hitpath ID,Offer Name,Advertiser,Sub Vertical,PUBID,Dataset,Drop,...,Hitpath_Offer_ID,Last_Drop_Date,Last_Drop_Revenue,Last_Drop_Delivered,Last_Drop_eCPM,Hitpath Offer ID,Pause Start,Pause End,Segment,count


In [32]:
# upcoming_light_with_bound = pd.merge(filtered_upcoming_light_traffic_offer_segment, last_7_drop_bound[['Hitpath Offer ID', 'Limited Traffic']], on = 'Hitpath Offer ID')
# upcoming_light_with_bound = upcoming_light_with_bound.sort_values(by='Last_Drop_eCPM', ascending=False)

# light_traffic_schedule = pd.DataFrame()

# for hitpath_offer_id, group in upcoming_light_with_bound.groupby('Hitpath Offer ID'):
#     delivered_bound = group['Limited Traffic'].iloc[0]
#     cumulative_count = 0
    
#     for index, row in group.iterrows():
#         row['Target Load Pct'] = f"{(row['count'] / delivered_bound) * 100:.2f}%"
#         if cumulative_count + row['count'] < delivered_bound:
#             row['need_swap'] = False
#             cumulative_count += row['count']
#         else:
#             row['need_swap'] = True
        
#         # Append the row to the result DataFrame
#         light_traffic_schedule = pd.concat([light_traffic_schedule, pd.DataFrame([row])], ignore_index=True)
                                     
# light_traffic_schedule = light_traffic_schedule.drop(columns=['Pause Start', 'Pause End', 'Hitpath Offer ID','Hitpath_Offer_ID', 'Last_Drop_Delivered', 'Last_Drop_Revenue', 'Creative', 'Job Name'])
# light_traffic_schedule.rename(columns={'count': 'Segment Size'}, inplace=True)
# light_traffic_schedule

In [33]:
# light_traffic_schedule

In [34]:
# light_traffic_swap = light_traffic_schedule[light_traffic_schedule['need_swap'] == True]
# light_traffic_swap

In [35]:
# Create an ExcelWriter object
filename = local_folder+'swap_report.xlsx'
writer=pd.ExcelWriter(filename, engine='xlsxwriter')
workbook=writer.book
worksheet = workbook.add_worksheet('P Drops')
worksheet2 = workbook.add_worksheet('AR Drops')
worksheet3 = workbook.add_worksheet('Underperforming Drops')
# worksheet4 = workbook.add_worksheet('Light Traffic')
bg_yellow =  workbook.add_format({'bg_color':   '#FCF3CF'})
bg_pink = workbook.add_format({'bg_color': '#FFC0CB'})
worksheet.set_column(0,14,15)
worksheet2.set_column(0,14,15)
worksheet3.set_column(0,13,15)
worksheet.conditional_format('B1:B1000', {'type':     'text',
                                       'criteria': 'containing',
                                       'value':    'Offer Wall',
                                       'format':   bg_yellow})
worksheet2.conditional_format('B1:B1000', {'type':     'text',
                                       'criteria': 'containing',
                                       'value':    'Offer Wall',
                                       'format':   bg_yellow})
alert_report.to_excel(writer, sheet_name='P Drops', startrow=0, startcol=0, index=False)
arschedule_swap_alert.to_excel(writer, sheet_name='AR Drops', startrow=0, startcol=0, index=False)
underperforming_swap.to_excel(writer, sheet_name='Underperforming Drops', startrow=0, startcol=0, index=False)
# light_traffic_schedule.to_excel(writer, sheet_name='Light Traffic', startrow=0, startcol=0, index=False)

# for row_num, swap_value in enumerate(light_traffic_schedule['need_swap'], start=1):  # start=1 to account for the header row
#     if swap_value:
#         worksheet4.conditional_format(f'A{row_num+1}:Z{row_num+1}', {'type': 'no_blanks', 'format': bg_pink})

workbook.close()

# Overlap Report

In [36]:
#overlap_report = pd.read_excel( local_folder+'sms-overlap.xlsx')
overlap_files =  [f for f in listdir(local_folder) if isfile(join(local_folder, f))& ('sms-overlap' in f)]
overlap_paths = ["{}/{}".format(local_folder,lexi_version) for lexi_version in overlap_files]
times = [os.path.getmtime(path) for path in overlap_paths]
# times_formatted = [time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(time_float)) for time_float in times]
most_recent_overlap = overlap_files[times.index(max(times))]
overlap_report_name = "{}/{}".format(local_folder,most_recent_overlap)
overlap_report = pd.read_excel( overlap_report_name,engine='openpyxl')

overlap_report['pub1_Dataset'] =  overlap_report['pub1_sc'].str.split('-',expand = True)[2]+'_'+overlap_report['pub1_dp_ds']+'_'+ overlap_report['pubid1'].astype('str')
overlap_report['pub2_Dataset'] =  overlap_report['pub2_sc'].str.split('-',expand = True)[2]+'_'+overlap_report['pub2_dp_ds']+'_'+ overlap_report['pubid2'].astype('str')
high_overlap = overlap_report[(overlap_report['pct_overlap'] > 0.1) & (overlap_report['pct_overlap'] <1)] 
high_overlap_schedule = high_overlap[['pub1_Dataset','pub2_Dataset','pct_overlap']].merge(All_merge, how = 'left', left_on = 'pub1_Dataset', right_on = 'Dataset',copy = False)
high_overlap_schedule = high_overlap_schedule.merge(All_merge[['Dataset','Hitpath ID','Date']],how = 'left', left_on = ['pub2_Dataset','Hitpath ID'], right_on = ['Dataset','Hitpath ID'], copy = False )
high_overlap_schedule['Offer Gap between Overlapped Datasets'] = abs(high_overlap_schedule['Date_x'] - high_overlap_schedule['Date_y'])
high_overlap_schedule = high_overlap_schedule.loc[(abs((high_overlap_schedule['Date_x'] - high_overlap_schedule['Date_y']).dt.days)<=2) & ((abs((high_overlap_schedule['Date_x'] - high_overlap_schedule['Date_y']).dt.days)>=0)),]
high_overlap_schedule = high_overlap_schedule.drop(['Dataset_x','Dataset_y'], axis = 1)
high_overlap_schedule = high_overlap_schedule[['Hitpath ID','pub1_Dataset', 'Date_x','DMA','Sub Vertical', 'Send Strategy','pub2_Dataset','Date_y','pct_overlap','Offer Gap between Overlapped Datasets']]
high_overlap_schedule = high_overlap_schedule.rename(columns = {'Date_x':'Date','Date_y':'Overlapped_pub_Date','pub1_Dataset':'Dataset','pub2_Dataset':'Overlapped_pub'})
high_overlap_schedule['Hitpath ID'] = high_overlap_schedule['Hitpath ID'].astype('int').astype('str')
high_overlap_schedule['pct_overlap'] =( high_overlap_schedule['pct_overlap']*100).round(2).astype('str') + '%'
high_overlap_schedule['Warning'] = "Please move the drops in " +high_overlap_schedule['Dataset'] +" with offer "+ high_overlap_schedule['Hitpath ID']+ " to 3 days away from "+ high_overlap_schedule['Overlapped_pub_Date'].astype('str') + " to avoid high overlap"
high_overlap_schedule.to_csv(local_folder+'Overlap_Accounts_Swap_Report.csv', index = False)

In [37]:
from datetime import datetime
today_date = datetime.now().strftime('%Y-%m-%d')
today_date

'2024-07-29'

In [38]:
current_ids

{}

In [39]:

toaddr =  ['sms@rxmg.com','offernotices@rxmg.com','prasad@rxmg.com']
#toaddr =  ['nina@rxmg.com']
os.chdir(filepath.output_folder) 
filename = ['swap_report.xlsx','SMS_upcoming_schedule.xlsx','AR_schedule.xlsx']
no_swap = len(alert_report)
subject_line = f"SMS Team Upcoming Schedule & Swap 'Report & Overlap Accounts Swap Report-{today_date}"
email_body = "Hi Team,\n\n▶ Here's the SMS Upcoming Schedule link: \nhttps://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=1189721992 \n\n▶ SWAP NOTICE:(swap report attached)\n"
if len(alert_report) >0: 
    email_body += "\n\t• Please Swap the Following P Drops: \n"
    for i in alert_report['Responsible DMA'].unique(): 
        email_body += "\t\t"+ i +":"+'\n' 
        dma = alert_report[alert_report['Responsible DMA'] == i]
        num_drops = len(dma)
        email_body += f"\t\t\t{num_drops} drops need to be swapped\n"
        #for i in range(0,len(dma)): 
            #row_dma = dma.iloc[i]

            #dma_send_strategy = row_dma['Send Strategy']
            #dma_offer = row_dma['Offer']
            #dma_date =row_dma['Date'].strftime('%Y-%m-%d')
            #dma_dataset = row_dma['Dataset']
            #dma_alert = row_dma['Alert']
            #email_body += f"  The Offer {dma_offer} sent on {dma_date} in {dma_dataset} need to be swapped/updated due to {dma_alert}\n"
    for i in alert_report['AM'].unique(): 
        email_body += "\t\t" + str(i) + ":" + '\n'  
        AM = alert_report[alert_report['AM'] == i]
        num_drops = len(AM)
        email_body += f"\t\t\t{num_drops} drops need to be swapped\n"

arschedule_swap_alert = arschedule_swap_alert.loc[arschedule_swap_alert['PUBID']!='nan',]
if len(arschedule_swap_alert) >0:
    email_body += "\n\t• Please Swap the Following AR Drops: \n"
    for i in arschedule_swap_alert['DMA'].unique(): 
        email_body += "\t\t"+i +":"+'\n' 
        AM = arschedule_swap_alert[arschedule_swap_alert['DMA'] == i]
        num_drops = len(AM)
        email_body += f"\t\t\t{num_drops} drops need to be swapped\n"
        #dataset_lyst =  [item for item in dma['New Offer Name'].tolist() if item != '']
        #dma_dataset = ", ".join(dataset_lyst)
        #email_body += f"\t{dma_dataset}\n"

underperforming_swap = underperforming_swap.loc[underperforming_swap['PUBID']!='nan',] 
if len(underperforming_swap) > 0: 
    email_body += "\n\t• Please Swap the Following Underperforming Drops: \n"
    for i in underperforming_swap['DMA'].unique(): 
        email_body += "\t\t" + i +":"+ "\n"
        AM = underperforming_swap[underperforming_swap['DMA'] == i]
        num_drops = len(AM)
        email_body += f"\t\t\t{num_drops} drops need to be swapped\n"  
    for i in underperforming_swap['AM'].unique(): 

        email_body += "\t\t" + str(i) +":"+ "\n"
        AM = underperforming_swap[underperforming_swap['AM'] == i]
        num_drops = len(AM)
        email_body += f"\t\t\t{num_drops} drops need to be swapped\n"

        
# if len(light_traffic_swap) >0: 
#     email_body += "\n\t• Please Swap the Following Drops Due to Light Traffic: \n"
#     for i in light_traffic_swap['DMA'].unique(): 
#         email_body += "\t\t"+ i +":"+'\n' 
#         dma = light_traffic_swap[light_traffic_swap['DMA'] == i]
#         num_drops = len(dma)
#         email_body += f"\t\t\t{num_drops} drops need to be swapped\n"
# else:
#     email_body += "\n\t•No drop exceed light traffic limit.\n"        
        
if current_ids:  # Check if current_ids is not empty
    email_body += "\n\t•Current week Loan offers exceed limit in following shortcode_DP.SV:\n"
    for (shortcode, dma), ids in current_ids.items():
        email_body += f"\t\t{dma} ({shortcode}): {ids}\n" 
else:
    email_body += "\n\t•No loan offers exceed the limit for the current week.\n"

if next_ids:  # Check if next_ids is not empty
    email_body += "\n\t•Next week Loan offers exceed limit in following shortcode_DP.SV:\n"
    for (shortcode, dma), ids in next_ids.items():
        email_body += f"\t\t{dma} ({shortcode}): {ids}\n"
else:
    email_body += "\n\t•No loan offers exceed the limit for the next week.\n"


email_body += "\n\n▶ AR NOTICE (AR Schedule Report attached):\n"
num_drops = len(ar_need_to_add)
if num_drops > 0:
    email_body += f"\n\t• {num_drops} AR Drops waiting be added in SS (swap report attached): \n"

if len(recent_update_ar)> 0:
    email_body += "\n\t• Please do peer check on the AR drops that recently added in SS: \n"
    for i in recent_update_ar['DMA'].unique(): 
        email_body += "\t\t"+i +":"+'\n' 
        num_drops = len(recent_update_ar[recent_update_ar['DMA'] == i])
        email_body += f"\t\t\t{num_drops} drops\n" 
""" 
email_body += "\n\nHere's the swap details from the Overlap Accounts Swap Report(attached) due to less than 3 days' offer gap between highly overlapped datasets: \n"
for i in high_overlap_schedule['DMA'].unique():
    email_body += "\t"+i +":"+'\n' 
    dma = high_overlap_schedule[high_overlap_schedule['DMA'] == i] 
    for i in range(0,len(dma)):
        row_dma = dma.iloc[i]
        warning = row_dma['Warning']
        
        email_body += f"  {warning}\n"
"""
email_body += '\n\nHave a nice day!\n\nThanks\nLili Guo\n\n'
 
for i in toaddr:
    send_email.send_email(filename,subject_line,email_body,i)



connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP 98e67ed59e1d1-2cdb7600054sm11061956a91.47 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP 98e67ed59e1d1-2cdb7600054sm11061956a91.47 - gsmtp'
connect: b'smtp.gmail.com ESMTP 98e67ed59e1d1-2cdb7600054sm11061956a91.47 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [52.38.54.186]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
r

reply: b'250 2.0.0 OK  1722287467 98e67ed59e1d1-2cdb7600054sm11061956a91.47 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1722287467 98e67ed59e1d1-2cdb7600054sm11061956a91.47 - gsmtp'
data: (250, b'2.0.0 OK  1722287467 98e67ed59e1d1-2cdb7600054sm11061956a91.47 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection 98e67ed59e1d1-2cdb7600054sm11061956a91.47 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection 98e67ed59e1d1-2cdb7600054sm11061956a91.47 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP d9443c01a7336-1fed7f2c2e2sm87415875ad.193 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP d9443c01a7336-1fed7f2c2e2sm87415875ad.193 - gsmtp'
connect: b'smtp.gmail.com ESMTP d9443c01a7336-1fed7f2c2e2sm87415875ad.193 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'25

reply: b'250 2.0.0 OK  1722287471 d9443c01a7336-1fed7f2c2e2sm87415875ad.193 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1722287471 d9443c01a7336-1fed7f2c2e2sm87415875ad.193 - gsmtp'
data: (250, b'2.0.0 OK  1722287471 d9443c01a7336-1fed7f2c2e2sm87415875ad.193 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection d9443c01a7336-1fed7f2c2e2sm87415875ad.193 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection d9443c01a7336-1fed7f2c2e2sm87415875ad.193 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP d9443c01a7336-1fed7ee4d26sm87831985ad.166 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP d9443c01a7336-1fed7ee4d26sm87831985ad.166 - gsmtp'
connect: b'smtp.gmail.com ESMTP d9443c01a7336-1fed7ee4d26sm87831985ad.166 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'25

reply: b'250 2.0.0 OK  1722287476 d9443c01a7336-1fed7ee4d26sm87831985ad.166 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1722287476 d9443c01a7336-1fed7ee4d26sm87831985ad.166 - gsmtp'
data: (250, b'2.0.0 OK  1722287476 d9443c01a7336-1fed7ee4d26sm87831985ad.166 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection d9443c01a7336-1fed7ee4d26sm87831985ad.166 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection d9443c01a7336-1fed7ee4d26sm87831985ad.166 - gsmtp'


In [40]:
from Monday_upload import *
# "SMS Upcoming Schedule"
filepath = local_folder+'SMS_upcoming_schedule.xlsx'
filename = 'SMS_upcoming_schedule.xlsx'
item_id  = reports_dict["SMS Upcoming Schedule"]
clean_file(item_id)
time.sleep(30)
add_file_monday(filepath, filename, item_id)
filepath = local_folder + 'AR_schedule.xlsx'
filename = 'AR_schedule.xlsx'
add_file_monday(filepath, filename, item_id)

Clear response: {"data":{"change_column_value":{"id":"7044687692"}},"account_id":6100183}
{"data":{"add_file_to_column":{"id":"1588373477"}},"account_id":6100183}
{"data":{"add_file_to_column":{"id":"1588374440"}},"account_id":6100183}


In [41]:
# "Swap Report"
filepath = local_folder + 'swap_report.xlsx'
filename = 'swap_report.xlsx'
item_id  = reports_dict["Swap Report"]
upload_file(filepath,filename, item_id)
email_body = email_body
add_update_to_item(item_id, email_body)


Clear response: {"data":{"change_column_value":{"id":"7044689311"}},"account_id":6100183}
{"data":{"add_file_to_column":{"id":"1588376060"}},"account_id":6100183}
{"data":{"create_update":{"id":"3335619929"}},"account_id":6100183}
